# 단어의 임베딩
- 빈도수 계산 : 빈도기반 - TF 상대빈도
- TDM : matrix TF를 행렬로 만든 것, 사전을 이용한 단순빈도
- TF-IDF : TF * IDF
- IDF : 역문서빈도

In [1]:
text = "John likes to watch movies. Mary likes movies too. \
    Mary also likes to watch football games."
words = text.replace('.', '').split()
words

['John',
 'likes',
 'to',
 'watch',
 'movies',
 'Mary',
 'likes',
 'movies',
 'too',
 'Mary',
 'also',
 'likes',
 'to',
 'watch',
 'football',
 'games']

In [2]:
import numpy as np
word_count = np.unique(words, return_counts = True)
word_count

(array(['John', 'Mary', 'also', 'football', 'games', 'likes', 'movies',
        'to', 'too', 'watch'], dtype='<U8'),
 array([1, 2, 1, 1, 1, 3, 2, 2, 1, 2]))

In [4]:
# 딕셔너리 TF 생성
word_to_cnt = {}
for word, cnt in zip(*word_count):
    word_to_cnt[word] = cnt

word_to_cnt

{np.str_('John'): np.int64(1),
 np.str_('Mary'): np.int64(2),
 np.str_('also'): np.int64(1),
 np.str_('football'): np.int64(1),
 np.str_('games'): np.int64(1),
 np.str_('likes'): np.int64(3),
 np.str_('movies'): np.int64(2),
 np.str_('to'): np.int64(2),
 np.str_('too'): np.int64(1),
 np.str_('watch'): np.int64(2)}

In [5]:
word_to_cnt['movies']

np.int64(2)

In [7]:
corpus = [
    "John likes to watch movies. Mary likes movies too.",
    "Mary also likes to watch football games."
]

# TDM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer()
tdm_array = vector.fit_transform(corpus).toarray()
tdm_array   # 2. 정의된 단어 순서대로 문장마다 몇 번 나타나는가

array([[0, 0, 0, 1, 2, 1, 2, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 0, 1, 0, 1]])

In [ ]:
vector.vocabulary_   # 1. 사전으로 정의된 단어 순서

{'john': 3,
 'likes': 4,
 'to': 7,
 'watch': 9,
 'movies': 6,
 'mary': 5,
 'too': 8,
 'also': 0,
 'football': 1,
 'games': 2}

In [12]:
tf_dic = vector.vocabulary_

In [13]:
import pandas as pd
tf_dic_sorted = dict(sorted(tf_dic.items(), key=lambda item: item[1]))
tf_dic_sorted

{'also': 0,
 'football': 1,
 'games': 2,
 'john': 3,
 'likes': 4,
 'mary': 5,
 'movies': 6,
 'to': 7,
 'too': 8,
 'watch': 9}

In [14]:
df = pd.DataFrame(tdm_array, columns=tf_dic_sorted.keys())
df

,also,football,games,john,likes,mary,movies,to,too,watch
0,0,0,0,1,2,1,2,1,1,1
1,1,1,1,0,1,1,0,1,0,1


# TF-IDF -> matrix

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
tfidf_array = tfidf_vec.fit_transform(corpus).toarray()
tfidf_array

array([[0.        , 0.        , 0.        , 0.32369906, 0.46062909,
        0.23031454, 0.64739811, 0.23031454, 0.32369906, 0.23031454],
       [0.44610081, 0.44610081, 0.44610081, 0.        , 0.3174044 ,
        0.3174044 , 0.        , 0.3174044 , 0.        , 0.3174044 ]])

In [19]:
tfidf_dic = tfidf_vec.vocabulary_
# tfidf_dic
tfidf_dic_sorted = dict(sorted(tfidf_dic.items(), key = lambda item: item[1]))
tfidf_dic_sorted

{'also': 0,
 'football': 1,
 'games': 2,
 'john': 3,
 'likes': 4,
 'mary': 5,
 'movies': 6,
 'to': 7,
 'too': 8,
 'watch': 9}

In [20]:
tfidf_dtm = pd.DataFrame(tfidf_array, columns=tfidf_dic_sorted.keys())
tfidf_dtm

,also,football,games,john,likes,mary,movies,to,too,watch
0,0.000000,0.000000,0.000000,0.323699,0.460629,0.230315,0.647398,0.230315,0.323699,0.230315
1,0.446101,0.446101,0.446101,0.000000,0.317404,0.317404,0.000000,0.317404,0.000000,0.317404


In [2]:
from gensim.models import Word2Vec
corpus = [
    "John likes to watch movies. Mary likes movies too.",
    "Mary also likes to watch football games."
]
word_list = []

In [3]:
for word in corpus:
    word_list.append(word.replace('.', '').split())

word_list

[['John', 'likes', 'to', 'watch', 'movies', 'Mary', 'likes', 'movies', 'too'],
 ['Mary', 'also', 'likes', 'to', 'watch', 'football', 'games']]

In [ ]:
model = Word2Vec(word_list, sg=0, vector_size=100, window=3, min_count=1)
model.wv.most_similar('likes', 'movies')   # 'likes', 'movies'와 유사한 정도

[('John', 0.17164471745491028),
 ('also', 0.06594578176736832),
 ('Mary', 0.008838453330099583),
 ('watch', -0.06765829026699066),
 ('games', -0.08544928580522537),
 ('football', -0.08948154747486115),
 ('too', -0.11860241740942001),
 ('to', -0.13643866777420044)]

In [8]:
model = Word2Vec(word_list, sg=1, vector_size=100, window=3, min_count=1)
model.wv.most_similar('likes', 'movies') 

[('John', 0.17164471745491028),
 ('also', 0.06594578176736832),
 ('Mary', 0.008853347972035408),
 ('watch', -0.06765829026699066),
 ('games', -0.08544928580522537),
 ('football', -0.08948154747486115),
 ('too', -0.11860241740942001),
 ('to', -0.13643862307071686)]

In [9]:
model = Word2Vec(word_list, sg=0, vector_size=100, window=3, min_count=1)
model.wv.most_similar('John', 'Mary') 

[('likes', 0.15334713459014893),
 ('football', 0.07839643210172653),
 ('also', 0.015055425465106964),
 ('too', 0.007465780712664127),
 ('movies', -0.006201202515512705),
 ('games', -0.07736953347921371),
 ('to', -0.12009607255458832),
 ('watch', -0.16032634675502777)]